In [1]:
import firebase_admin
import json
import os
import time
from dotenv import load_dotenv
from firebase_admin import credentials
from firebase_admin import firestore
from google.cloud import storage

In [2]:
# uploading the environment variables and get the API key
load_dotenv()
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")  # Borrar esto cuando se completen los embeddings desde VERTEX
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")

# Asegurar que Vertex AI use las credenciales correctas
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../../bubbo-dfba0-47e395cdcdc7.json"

In [3]:
# Ruta a las credenciales de Firebase para la app principal
cred_default_path = "../../../bubbo-dfba0-47e395cdcdc7.json"

# Inicializar Firebase
if not firebase_admin._apps:
    # Inicializar la app por defecto
    cred_default = credentials.Certificate(cred_default_path)
    default_app = firebase_admin.initialize_app(cred_default, name="default")

# Conectar Firestore con la app inicializada
db = firestore.client(app=default_app)  

In [5]:
import json
from google.cloud import storage
from google.api_core.exceptions import NotFound, Forbidden

# Configuración del bucket y ruta de embeddings
BUCKET_NAME = "embeddings_bucket_backup_europe"  
EMBEDDINGS_FOLDER = "2025-03-07T08:59:10_70998/all_namespaces/kind_embeddings/"  

# Inicializa el cliente de Google Cloud Storage
storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)

def list_embedding_files():
    """Lista los archivos disponibles en la carpeta de embeddings del bucket"""
    try:
        blobs = list(bucket.list_blobs(prefix=EMBEDDINGS_FOLDER))
        if not blobs:
            print(f"⚠️ No se encontraron archivos en gs://{BUCKET_NAME}/{EMBEDDINGS_FOLDER}")
            return []
        return [blob.name for blob in blobs]  # Lista todos los archivos en la carpeta
    except NotFound:
        print(f"❌ ERROR: El bucket '{BUCKET_NAME}' no existe.")
        return []
    except Forbidden:
        print(f"🚫 ERROR: No tienes permisos para acceder al bucket '{BUCKET_NAME}'.")
        return []

def download_embedding(file_name):
    """Descarga un archivo desde GCS y devuelve su contenido como texto"""
    try:
        blob = bucket.blob(file_name)
        content = blob.download_as_text()
        return content  # Los archivos parecen ser binarios, revisa si hay que decodificarlos
    except NotFound:
        print(f"❌ ERROR: El archivo {file_name} no existe en el bucket.")
        return None

def retrieve_some_embeddings(num_files=5):
    """Recupera y muestra algunos embeddings almacenados en GCS"""
    print(f"🔍 Buscando embeddings en 'gs://{BUCKET_NAME}/{EMBEDDINGS_FOLDER}'...")

    files = list_embedding_files()
    if not files:
        print("⚠️ No se encontraron archivos de embeddings.")
        return

    print(f"✅ {len(files)} archivos encontrados. Descargando los primeros {num_files}...")

    retrieved_embeddings = []
    for file_name in files[:num_files]:  # Descarga solo algunos archivos
        data = download_embedding(file_name)
        if data:
            retrieved_embeddings.append(data)
            print(f"📥 Descargado: {file_name} -> {len(data)} bytes")

    print("\n✅ Embeddings recuperados con éxito.")
    return retrieved_embeddings

# Llamada para recuperar embeddings
retrieved = retrieve_some_embeddings(5)


🔍 Buscando embeddings en 'gs://embeddings_bucket_backup_europe/2025-03-07T08:59:10_70998/all_namespaces/kind_embeddings/'...
✅ 1025 archivos encontrados. Descargando los primeros 5...


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x95 in position 30: invalid start byte